In [ ]:
# Import necessary libraries

import pandas as pd # For data manipulation and analysis
import numpy as np # For numerical operations
import pickle # For serializing and deserializing Python objects
from modelling_utils import train_knn_classifier, evaluate_model, plot_model_comparison

In [ ]:
# Load features from a pickle file
feature_dict_path = "/home/suraj/Repositories/FM-extractors-radiomics/evaluation/features/luna.pkl" # Path to the pickle file containing features
with open(feature_dict_path, 'rb') as file: # Open the file in read binary mode
    data = pickle.load(file) # Load the data from the pickle file

In [ ]:
# Store test accuracies for each model
test_accuracies_dict = {} # Initialize an empty dictionary to store test accuracies

# Iterate through each model's features
for model_name, values in data.items(): # Loop through each model and its corresponding features
    # Skip MedImageInsightExtractor
    if model_name == "MedImageInsightExtractor": # Skip the MedImageInsightExtractor model
        continue

    # Extract paths and labels for train, val, and test sets
    train_labels = [v["row"]["malignancy"] for v in values["train"]] # Extract malignancy labels for the training set
    val_labels = [v["row"]["malignancy"] for v in values["val"]] # Extract malignancy labels for the validation set
    test_labels = [v["row"]["malignancy"] for v in values["test"]] # Extract malignancy labels for the test set
    
    # Stack features
    train_items = np.vstack([v["feature"] for v in values["train"]]) # Stack features for the training set
    val_items = np.vstack([v["feature"] for v in values["val"]]) # Stack features for the validation set
    test_items = np.vstack([v["feature"] for v in values["test"]]) # Stack features for the test set

    # Train model with hyperparameter optimization
    best_model, study = train_knn_classifier(train_items, train_labels, val_items, val_labels, n_trials=10)
    
    # Evaluate on test set
    test_accuracies_dict[model_name] = evaluate_model(best_model, test_items, test_labels)

In [ ]:
# Plot test accuracies
fig = plot_model_comparison(test_accuracies_dict)
fig.show() # Show the plot